# Figures 5, 6, 7, 8, 9, and 13

## Load Libraries

In [0]:
import numpy as np
import xarray as xr
import geopandas as gpd
import matplotlib.pyplot as plt

import rasterio
import rioxarray
import contextily as ctx
import shapely
import pandas as pd
from shapely.geometry import LineString, Point, MultiLineString, Polygon, MultiPolygon
from scipy import interpolate
from itertools import islice
from fiona import path
from rasterio.transform import xy
import concurrent.futures
from matplotlib.lines import Line2D
from matplotlib.patches import Patch, Polygon

from matplotlib.colors import Normalize, LinearSegmentedColormap
from matplotlib import cm
from matplotlib.colorbar import ColorbarBase

from shapely import LineString
from shapely.geometry import shape as shp_shape
from shapely.ops import unary_union, linemerge, polygonize
from rasterio import features
import matplotlib.patches as mpatches
from matplotlib.legend_handler import HandlerTuple, HandlerBase


In [0]:

difference_threshold = 0.33

THRESH_MIN = 0.33   # keep >= 0.5 ft
THRESH_MAX = 3.5   # drop outliers > threshold in ft  (adjust if you want)
MIN_AREA_M2 = 20.0e5  # drop polygons smaller than this area (in m^2); set to 0 to keep all, used in mapping the boundary of the transition zone
SIMPLIFY_TOL = 10

data_dir1 = '/Volumes/lwi/default/transition-zone-data/pilot_reanalysis/output_data'
data_dir_pluvial = '/Volumes/lwi/default/transition-zone-data/pilot_reanalysis/prod_rerun_5k/tropical_pluvial_rasters'

## Functions

In [0]:

def extract_raster_along_line(raster_dataset, line_feature, num_points=1000):
    """
    Extract raster values along a line feature and calculate distances.
    
    Parameters:
    -----------
    raster_dataset : xarray.Dataset or xarray.DataArray
        The raster data
    line_feature : GeoSeries or GeoDataFrame with LineString or MultiLineString geometry
        The line along which to extract values
    num_points : int, optional
        Number of points to sample along the line
        
    Returns:
    --------
    tuple : (distances, values)
        distances: numpy array of distances along the line
        values: numpy array of raster values at each point
    """
    # Get the first line if multiple lines are present
    if isinstance(line_feature, gpd.GeoDataFrame):
        line = line_feature.geometry.iloc[0]
    else:
        line = line_feature.geometry
    
    # Handle MultiLineString by converting to LineString
    if isinstance(line, MultiLineString):
        # Convert MultiLineString to a single LineString by merging all segments
        # This assumes the segments can be meaningfully connected
        linestrings = list(line.geoms)
        coords = []
        for ls in linestrings:
            coords.extend(list(ls.coords))
        line = LineString(coords)
    elif not isinstance(line, LineString):
        raise ValueError("Input must be a LineString or MultiLineString geometry")
    
    # Create evenly spaced points along the line
    distances = np.linspace(0, line.length, num_points)
    points = [line.interpolate(distance) for distance in distances]
    
    # Get coordinates of points
    coords = [(point.x, point.y) for point in points]
    
    # Extract values from raster at each point
    if isinstance(raster_dataset, xr.Dataset):
        # Select the first data variable if it's a dataset
        var_name = list(raster_dataset.data_vars)[0]
        da = raster_dataset[var_name]
    else:
        da = raster_dataset
    
    # Get the raster values at each point
    values = []
    for x, y in coords:
        # Convert x,y to raster indices
        # This assumes the xarray has coordinates with names 'x' and 'y'
        # Adjust if your coordinate names are different (e.g., 'lon', 'lat')
        try:
            # Method 1: Using xarray's sel method with nearest neighbor
            val = da.sel(x=x, y=y, method="nearest").values.item()
        except (KeyError, ValueError):
            try:
                # Method 2: Try with different coordinate names
                val = da.sel(lon=x, lat=y, method="nearest").values.item()
            except (KeyError, ValueError):
                # If coordinate names are unknown, just use NaN
                val = np.nan
        values.append(val)
    
    return distances, np.array(values)

def ensure_connected_linestring(multiline):
    """
    Convert a MultiLineString to a single LineString with proper connectivity
    between segments.
    
    Parameters:
    -----------
    multiline : MultiLineString or LineString
        Input line geometry
        
    Returns:
    --------
    LineString
        A properly connected LineString
    """
    if isinstance(multiline, LineString):
        return multiline
    
    # Extract all segments from the MultiLineString
    segments = list(multiline.geoms)
    if not segments:
        raise ValueError("Empty MultiLineString provided")
    
    # Start with the first segment
    result_coords = list(segments[0].coords)
    segments_used = [0]
    
    # Keep track of both ends of our working LineString
    start_point = Point(result_coords[0])
    end_point = Point(result_coords[-1])
    
    # Keep connecting segments until all are used
    while len(segments_used) < len(segments):
        best_segment = None
        best_distance = float('inf')
        best_is_reversed = False
        best_idx = -1
        
        # Find the closest segment to either end of our working LineString
        for i, segment in enumerate(segments):
            if i in segments_used:
                continue
                
            seg_start = Point(segment.coords[0])
            seg_end = Point(segment.coords[-1])
            
            # Check distances to both ends of our working LineString
            dist_start_to_start = start_point.distance(seg_start)
            dist_start_to_end = start_point.distance(seg_end)
            dist_end_to_start = end_point.distance(seg_start)
            dist_end_to_end = end_point.distance(seg_end)
            
            # Find the minimum distance and corresponding configuration
            min_dist = min(dist_start_to_start, dist_start_to_end, 
                          dist_end_to_start, dist_end_to_end)
            
            if min_dist < best_distance:
                best_distance = min_dist
                best_segment = segment
                best_idx = i
                
                # Determine if we need to reverse the segment and where to connect it
                if min_dist == dist_start_to_start:
                    # Connect to start, reverse segment
                    best_is_reversed = True
                    connect_to_start = True
                elif min_dist == dist_start_to_end:
                    # Connect to start, don't reverse
                    best_is_reversed = False
                    connect_to_start = True
                elif min_dist == dist_end_to_start:
                    # Connect to end, don't reverse
                    best_is_reversed = False
                    connect_to_start = False
                else:  # min_dist == dist_end_to_end
                    # Connect to end, reverse segment
                    best_is_reversed = True
                    connect_to_start = False
        
        if best_segment is None:
            # This shouldn't happen if the MultiLineString is valid
            break
        
        # Get coords from the best segment
        new_coords = list(best_segment.coords)
        if best_is_reversed:
            new_coords.reverse()
        
        # Add to result_coords
        if connect_to_start:
            # Remove the first point which would be a duplicate
            result_coords = new_coords + result_coords[1:]
            start_point = Point(result_coords[0])
        else:
            # Remove the first point which would be a duplicate
            result_coords = result_coords[:-1] + new_coords
            end_point = Point(result_coords[-1])
        
        segments_used.append(best_idx)
    
    return LineString(result_coords)

def group_and_average(xs, threshold=2.5):
    xs_sorted = sorted(xs)
    groups = []
    current_group = [xs_sorted[0]]
    for x in xs_sorted[1:]:
        if abs(x - current_group[-1]) <= threshold:
            current_group.append(x)
        else:
            groups.append(current_group)
            current_group = [x]
    groups.append(current_group)
    return [sum(g)/len(g) for g in groups]   

# ---------- helpers ----------
def set_integer_yticks(ax, nbins=5):
    """Set y-axis ticks to integers only with controlled spacing."""
    from matplotlib.ticker import MaxNLocator
    ax.yaxis.set_major_locator(MaxNLocator(integer=True, nbins=nbins))

def all_level_crossings(x, y, level=0.5):
    """Return sorted list of x where y crosses 'level' (linear interp)."""
    x = np.asarray(x); y = np.asarray(y)
    d = y - level
    xs = []
    sign_change = (np.sign(d[:-1]) * np.sign(d[1:]) < 0) | (d[:-1] == 0) | (d[1:] == 0)
    idxs = np.where(sign_change)[0]
    for i in idxs:
        x0, x1 = x[i], x[i+1]
        y0, y1 = y[i], y[i+1]
        if d[i] == 0:
            xs.append(float(x0))
        elif d[i+1] == 0:
            xs.append(float(x1))
        elif y1 != y0:
            t = (level - y0) / (y1 - y0)
            if 0 <= t <= 1:
                xs.append(float(x0 + t * (x1 - x0)))
    # include flat segments exactly at level
    flat = np.where((d[:-1] == 0) & (d[1:] == 0))[0]
    for i in flat:
        xs.extend([float(x[i]), float(x[i+1])])
    xs = sorted(xs)
    uniq = []
    for v in xs:
        if not uniq or abs(v - uniq[-1]) > 1e-6:
            uniq.append(v)
    return uniq

def draw_cftz(left_ax, right_ax, x_left, x_right, mode="between", label="CFTZ"):
    """
    Draw verticals at x_left and x_right on both axes.
    mode="between": dotted span between x_left..x_right
    mode="to_right": dotted span from x_right..right edge
    Label is centered over the span and placed just BELOW the dotted line.
    """
    for ax in (left_ax, right_ax):
        ax.axvline(x_left,  color='black', lw=1.0)
        ax.axvline(x_right, color='black', lw=1.0)

        ymin, ymax = ax.get_ylim()
        yspan = ymax - ymin
        yline = ymax - 0.03 * yspan   # a hair below top
        # span endpoints
        if mode == "between":
            x0, x1 = x_left, x_right
        else:  # to_right
            # use current axis max
            xlim_max = ax.get_xlim()[1]
            x0, x1 = x_right, xlim_max

        ax.hlines(yline, x0, x1, linestyles=':', colors='black', lw=1.2)

        # centered label, BELOW the dotted line
        xmid = 0.5 * (x0 + x1)
        ax.text(xmid, yline - 0.02 * yspan, label,
                ha='center', va='top', fontsize=10, color='black', clip_on=False)
        
# ---------------- helper: draw transect with km ticks/labels ----------------
def overlay_transect(ax, line: LineString, km_ticks=(0,20,40,60,80, 100, 120),
                     line_lw=3.0, dot_size=22, label_fontsize=9,
                     ensure_north_start=False, label_position='center'):
    """
    Draw the Amite transect on ax with markers/labels at given km distances.
    - ensure_north_start=True will flip the line so that the 0-km end is the
      northernmost point (top of map). Leave False to keep original direction.
    - label_position: 'left', 'center', or 'right' to position the label relative to the symbol.
    """
    # If requested, flip so that "0" is the northern (top) end
    if ensure_north_start:
        x0, y0 = list(line.coords)[0]
        x1, y1 = list(line.coords)[-1]
        if y0 < y1:  # start is south of end; reverse so 0 is at the top
            line = LineString(list(line.coords)[::-1])

    # draw the line (bold white)
    xs, ys = np.array(line.coords.xy[0]), np.array(line.coords.xy[1])
    ax.plot(xs, ys, color='black', lw=line_lw, zorder=10, solid_capstyle='round')

    # total length (meters) and valid tick distances
    Lm = line.length
    tick_m = [k*1000 for k in km_ticks if k*1000 <= Lm]

    # interpolate points, draw dots and labels
    for idx, (k, dm) in enumerate(zip(km_ticks, [k*1000 for k in km_ticks])):
        if dm > Lm:
            continue
        pt = line.interpolate(dm)
        px, py = pt.x, pt.y
        # dot: white fill, black edge
        ax.scatter([px], [py], s=dot_size, facecolor='white',
                   edgecolor='black', linewidths=1.2, zorder=11)
        # label position adjustment
        if label_position == 'left':
            ha = 'right'
            offset = -0.012
        elif label_position == 'right':
            ha = 'left'
            offset = 0.012
        else:  # center
            ha = 'center'
            offset = 0

        # label slightly above the dot
        ymin, ymax = ax.get_ylim()
        offs = offset * (ymax - ymin)
        # Add 'km' to first and last tick label
        if idx == 0 or idx == len(km_ticks) - 1:
            label = f"{k} km"
        else:
            label = f"{k}"
        ax.text(px + offs, py + offs, label, ha=ha, va='bottom',
                fontsize=label_fontsize, color='white',
                zorder=12,
                path_effects=[__import__('matplotlib.patheffects').patheffects.withStroke(
                    linewidth=2, foreground='black')])
        
def mask_to_outline_gdf(mask01, ref_da, min_area_m2=MIN_AREA_M2):
    """Polygonize mask==1, dissolve, optional area filter, return boundaries as GeoDataFrame."""
    if mask01.sum() == 0:
        return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    geoms = [shp_shape(g) for g, val in features.shapes(mask01, transform=ref_da.rio.transform()) if int(val) == 1]
    if not geoms:
        return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    # dissolve and clean
    dissolved = unary_union(geoms).buffer(0)
    polys = gpd.GeoDataFrame(geometry=[dissolved], crs=ref_da.rio.crs).explode(index_parts=False).reset_index(drop=True)

    # optional area filter (works if CRS is projected in meters; EPSG:26915 is)
    if min_area_m2 and min_area_m2 > 0:
        polys = polys[polys.geometry.area >= min_area_m2]
        if polys.empty:
            return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    # boundaries
    return gpd.GeoDataFrame(geometry=polys.boundary, crs=ref_da.rio.crs)



# Inputs already in your session:
# diff_10, diff_50, diff_100, diff_500  (xarray/rioxarray DataArrays, same grid/CRS)



def make_band_mask(da, lo=THRESH_MIN, hi=THRESH_MAX):
    """Boolean mask for lo <= da <= hi (finite only), returned as uint8 0/1."""
    v = da.values
    m = np.isfinite(v) & (v >= lo) & (v <= hi)
    return m.astype("uint8")

def compound_masks(prev_mask, new_mask):
    """Ensure monotonic growth: logical OR with previous."""
    return new_mask if prev_mask is None else ((prev_mask > 0) | (new_mask > 0)).astype("uint8")



def mask_to_outline_gdf(mask01, ref_da, min_area_m2=MIN_AREA_M2, simplify_tol=SIMPLIFY_TOL):
    if mask01.sum() == 0:
        return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    geoms = [shp_shape(g) for g, val in features.shapes(mask01, transform=ref_da.rio.transform()) if int(val) == 1]
    if not geoms:
        return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    # dissolve and clean
    dissolved = unary_union(geoms).buffer(0)
    dissolved = _remove_small_inners(dissolved, 20e5)
    # Remove small speckles (area filter)
    if dissolved.geom_type == "MultiPolygon":
        dissolved = [p for p in dissolved.geoms if p.area >= min_area_m2]
        if not dissolved:
            return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)
        dissolved = unary_union(dissolved)
    elif dissolved.area < min_area_m2:
        return gpd.GeoDataFrame(geometry=[], crs=ref_da.rio.crs)

    # Simplify geometry
    simplified = dissolved.simplify(simplify_tol, preserve_topology=True)
    polys = gpd.GeoDataFrame(geometry=[simplified], crs=ref_da.rio.crs).explode(index_parts=False).reset_index(drop=True)

    return gpd.GeoDataFrame(geometry=polys.boundary, crs=ref_da.rio.crs)

def save_geojson(gdf, path):
    if gdf.empty:
        print(f"[save] {path}: empty, skipped.")
    else:
        gdf.to_file(path, driver="GeoJSON")
        print(f"[save] wrote {path} ({len(gdf)} parts)")



def _remove_small_inners(geom, min_area_m2):
    if geom.is_empty:
        return geom
    if geom.geom_type == "Polygon":
        # keep only interior rings that form polygons above threshold
        new_interiors = []
        for ring in geom.interiors:
            poly = Polygon(ring)
            if poly.area >= min_area_m2:
                new_interiors.append(ring)
        return Polygon(geom.exterior, new_interiors)

    elif geom.geom_type == "MultiPolygon":
        return MultiPolygon([_remove_small_inners(p, min_area_m2) for p in geom.geoms])
    else:
        return geom
    
def get_transition_polys(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Robustly select the 'Transition' zone polygons by:
      A) name-like columns containing 'transition' (case-insensitive)
      B) style/color columns (fill/stroke/color/colour) containing 'yellow'
         or common yellow-ish hex/RGB hints
    """
    if gdf.empty:
        return gdf

    # A) Token search in any string/object column
    token_mask = np.zeros(len(gdf), dtype=bool)
    TOKENS = ("transition", "transitional")
    for col in gdf.columns:
        if gdf[col].dtype == "object":
            s = gdf[col].astype(str).str.lower()
            for t in TOKENS:
                token_mask |= s.str.contains(t, regex=False)

    # B) Style/color hints
    style_mask = np.zeros(len(gdf), dtype=bool)
    STYLE_COLS = [c for c in gdf.columns if c.lower() in {"fill","stroke","color","colour"}]
    # yellow-ish values to catch common encodings
    YELLOW_HINTS = (
        "yellow",          # named
        "#ff0", "#ffff00", # classic hex
        "#ffcc00", "#ffd700", "#fdd835", "#ffc107",  # common yellows
        "255,255,0", "rgb(255,255,0)"                # RGB text
    )
    for sc in STYLE_COLS:
        s = gdf[sc].astype(str).str.lower()
        m = np.zeros(len(gdf), dtype=bool)
        for y in YELLOW_HINTS:
            m |= s.str.contains(y, regex=False)
        style_mask |= m

    sel = token_mask | style_mask
    trans = gdf.loc[sel].copy()

    # If multiple parts, dissolve to a single feature for a clean boundary
    if not trans.empty:
        trans = trans.dissolve().reset_index(drop=True)

    return trans

def _remove_small_inners(geom, min_area_m2):
    if geom.is_empty:
        return geom
    if geom.geom_type == "Polygon":
        new_interiors = []
        for ring in geom.interiors:
            poly = Polygon(ring)
            if poly.area >= min_area_m2:
                new_interiors.append(ring)
        return Polygon(geom.exterior, new_interiors)
    elif geom.geom_type == "MultiPolygon":
        return MultiPolygon([_remove_small_inners(p, min_area_m2) for p in geom.geoms])
    else:
        return geom

def simplify_transition_gdf(
    gdf, min_area_m2=MIN_AREA_M2, simplify_tol=SIMPLIFY_TOL):
    if gdf.empty:
        return gpd.GeoDataFrame(geometry=[], crs=gdf.crs)
    geoms = list(gdf.geometry)
    dissolved = unary_union(geoms).buffer(0)
    dissolved = _remove_small_inners(dissolved, 20e5)
    if dissolved.geom_type == "MultiPolygon":
        dissolved = [p for p in dissolved.geoms if p.area >= min_area_m2]
        if not dissolved:
            return gpd.GeoDataFrame(geometry=[], crs=gdf.crs)
        dissolved = unary_union(dissolved)
    elif dissolved.area < min_area_m2:
        return gpd.GeoDataFrame(geometry=[], crs=gdf.crs)
    simplified = dissolved.simplify(simplify_tol, preserve_topology=True)
    polys = gpd.GeoDataFrame(geometry=[simplified], crs=gdf.crs).explode(index_parts=False).reset_index(drop=True)
    return gpd.GeoDataFrame(geometry=polys.boundary, crs=gdf.crs)

## Load Data

In [0]:
#Load rasters origin
# ally created by Purdue, but which are not correct for comparison.
combined_fig_10 = rioxarray.open_rasterio(f'{data_dir1}/Spline_Tension_T10_joint_depth_adj.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_fig_50 = rioxarray.open_rasterio(f'{data_dir1}/Spline_Tension_T50_joint_depth_adj.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_fig_100 = rioxarray.open_rasterio(f'{data_dir1}/Spline_Tension_T100_joint_depth_adj.tif').rio.reproject("EPSG:26915").sel(band = 1)

combined_fig_10_1 = rioxarray.open_rasterio(f'{data_dir1}/Spline_T10_joint_depth1.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_fig_50_1 = rioxarray.open_rasterio(f'{data_dir1}/Spline_T50_joint_depth1.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_fig_100_1 = rioxarray.open_rasterio(f'{data_dir1}/Spline_T100_joint_depth1.tif').rio.reproject("EPSG:26915").sel(band = 1)

In [0]:


#Load TC only Compound Flood Rasters
compound_10 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_compound_0010_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
compound_50 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_compound_0050_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
compound_100 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_compound_0100_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
compound_500 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_compound_0500_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)

#Load TC and non TC combined compound Flood rasters
combined_10 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_combined_compound_0010_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_50 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_combined_compound_0050_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_100 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_combined_compound_0100_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
combined_500 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_combined_compound_0500_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)

combined_10 = (combined_10.rio.reproject_match(compound_10))
combined_50 = (combined_50.rio.reproject_match(compound_50))
combined_100 = (combined_100.rio.reproject_match(compound_100))
combined_500 = (combined_500.rio.reproject_match(compound_500))

#Load surge only, riverine only, and pluvial only rasters.
surge_10 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_surge_only_0010_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
surge_50 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_surge_only_0050_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
surge_100 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_surge_only_0100_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)
surge_500 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_surge_only_0500_YR_WSE_no_aleatory_bartlett_edits.tif').rio.reproject("EPSG:26915").sel(band = 1)

riverine_10 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_riverine_only_0010_YR_WSE_revised.tif').rio.reproject("EPSG:26915").sel(band = 1)
riverine_50 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_riverine_only_0050_YR_WSE_revised.tif').rio.reproject("EPSG:26915").sel(band = 1)
riverine_100 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_riverine_only_0100_YR_WSE_revised.tif').rio.reproject("EPSG:26915").sel(band = 1)
riverine_500 = rioxarray.open_rasterio(f'{data_dir1}/EJPMOS_tropical_riverine_only_0500_YR_WSE_revised.tif').rio.reproject("EPSG:26915").sel(band = 1)

pluvial_10 = rioxarray.open_rasterio(f'{data_dir_pluvial}/EJPMOS_tropical_pluvial_0010_YR_WSE_os_no_aleatory.tif').rio.reproject("EPSG:26915").sel(band = 1)
pluvial_50 = rioxarray.open_rasterio(f'{data_dir_pluvial}/EJPMOS_tropical_pluvial_0050_YR_WSE_os_no_aleatory.tif').rio.reproject("EPSG:26915").sel(band = 1)
pluvial_100 = rioxarray.open_rasterio(f'{data_dir_pluvial}/EJPMOS_tropical_pluvial_0100_YR_WSE_os_no_aleatory.tif').rio.reproject("EPSG:26915").sel(band = 1)
pluvial_500 = rioxarray.open_rasterio(f'{data_dir_pluvial}/EJPMOS_tropical_pluvial_0500_YR_WSE_os_no_aleatory.tif').rio.reproject("EPSG:26915").sel(band = 1)

#load elevation and Amite river line data.
grade_elev = rioxarray.open_rasterio(f'{data_dir1}/amite_ground_elev_prod.tif').rio.reproject("EPSG:26915").sel(band = 1)
amite_line = gpd.read_file(f'{data_dir1}/amite_line_pass_manchac_centered_userpoints.json').to_crs('EPSG:26915')
#amite_line_extended = gpd.read_file(f'{data_dir1}/amite_line_extended.json').to_crs('EPSG:26915')

### Take greater of the WSE or ground elevation
Also only take the surge values where the pluvial is valid to remove the spurious surge conditions are the boundary of the model where the surge condition is applied.

In [0]:
surge_10 = np.fmax(surge_10, grade_elev)
surge_50 = np.fmax(surge_50, grade_elev)
surge_100 = np.fmax(surge_100, grade_elev)
surge_500 = np.fmax(surge_500, grade_elev)


combined_10 =  np.fmax(combined_10, grade_elev)
combined_50 =  np.fmax(combined_50, grade_elev)
combined_100 =  np.fmax(combined_100, grade_elev)
combined_500 =  np.fmax(combined_500, grade_elev)

compound_10 =  np.fmax(compound_10, grade_elev)
compound_50 =  np.fmax(compound_50, grade_elev)
compound_100 =  np.fmax(compound_100, grade_elev)
compound_500 =  np.fmax(compound_500, grade_elev)

surge_10 = surge_10.where(~np.isnan(pluvial_10), np.nan)
surge_50 = surge_50.where(~np.isnan(pluvial_50), np.nan)
surge_100 = surge_100.where(~np.isnan(pluvial_100), np.nan)
surge_500 = surge_500.where(~np.isnan(pluvial_500), np.nan)

### Calculate differences in the flood surfaces

In [0]:


pluvial_surge_10 = np.fmax(pluvial_10, surge_10)
pluvial_surge_riverine_10 = np.fmax(pluvial_surge_10, riverine_10)
pluvial_surge_50 = np.fmax(pluvial_50, surge_50)
pluvial_surge_riverine_50 = np.fmax(pluvial_surge_50, riverine_50)
pluvial_surge_100 = np.fmax(pluvial_100, surge_100)
pluvial_surge_riverine_100 = np.fmax(pluvial_surge_100, riverine_100)
pluvial_surge_500 = np.fmax(pluvial_500, surge_500)
pluvial_surge_riverine_500 = np.fmax(pluvial_surge_500, riverine_500)

riverine_surge_10 = np.fmax(riverine_10, surge_10)
riverine_surge_50 = np.fmax(riverine_50, surge_50)
riverine_surge_100 = np.fmax(riverine_100, surge_100)
riverine_surge_500 = np.fmax(riverine_500, surge_500)

diff_10 = compound_10 - pluvial_surge_riverine_10
diff_50 = compound_50 - pluvial_surge_riverine_50
diff_100 = compound_100 - pluvial_surge_riverine_100
diff_500 = compound_500 - pluvial_surge_riverine_500

diff_10 = diff_10.where(diff_10 >= 0, 0)
diff_50 = diff_50.where(diff_50 >= 0, 0)
diff_100 = diff_100.where(diff_100 >= 0, 0)
diff_500 = diff_500.where(diff_500 >= 0, 0)

diff_10_nopluv = compound_10 - riverine_surge_10
diff_50_nopluv = compound_50 - riverine_surge_50
diff_100_nopluv = compound_100 - riverine_surge_100
diff_500_nopluv = compound_500 - riverine_surge_500

diff_10_nopluv = diff_10_nopluv.where(diff_10_nopluv >= 0, 0)
diff_50_nopluv = diff_50_nopluv.where(diff_50_nopluv >= 0, 0)
diff_100_nopluv = diff_100_nopluv.where(diff_100_nopluv >= 0, 0)
diff_500_nopluv = diff_500_nopluv.where(diff_500_nopluv >= 0, 0)

compound_depth_10 = compound_10 - grade_elev
compound_depth_50 = compound_50 - grade_elev
compound_depth_100 = compound_100 - grade_elev
compound_depth_500 = compound_500 - grade_elev

combined_depth_10 = combined_10 - grade_elev
combined_depth_50 = combined_50 - grade_elev
combined_depth_100 = combined_100 - grade_elev
combined_depth_500 = combined_500 - grade_elev

#combined_depth_10 = combined_depth_10.where(combined_depth_10 != 0)
#combined_depth_50 = combined_depth_10.where(combined_depth_50 != 0)
#combined_depth_100 = combined_depth_10.where(combined_depth_100 != 0)
#combined_depth_500 = combined_depth_10.where(combined_depth_500 != 0)

#combined_depth_10 = combined_depth_10.where((combined_depth_10 >= 0)|(combined_depth_10.isnull()), 0)
#combined_depth_50 = combined_depth_50.where((combined_depth_50 >= 0)|(combined_depth_50.isnull()), 0)
#combined_depth_100 = combined_depth_100.where((combined_depth_100 >= 0)|(combined_depth_100.isnull()), 0)
#combined_depth_500 = combined_depth_500.where((combined_depth_500 >= 0)|(combined_depth_500.isnull()), 0)

diff_10_pluv = pluvial_surge_riverine_10 - riverine_surge_10
diff_50_pluv = pluvial_surge_riverine_50 - riverine_surge_50
diff_100_pluv = pluvial_surge_riverine_100 - riverine_surge_100
diff_500_pluv = pluvial_surge_riverine_500 - riverine_surge_500

nt_contribution_10 = combined_10 - compound_10
nt_contribution_50 = combined_50 - compound_50
nt_contribution_100 = combined_100 - compound_100
nt_contribution_500 = combined_500 - compound_500

nt_contribution_10 = nt_contribution_10.where((nt_contribution_10 >= 0)|(nt_contribution_10.isnull()), 0)
nt_contribution_50 = nt_contribution_50.where((nt_contribution_50 >= 0)|(nt_contribution_50.isnull()), 0)
nt_contribution_100 = nt_contribution_100.where((nt_contribution_100 >= 0)|(nt_contribution_100.isnull()), 0)
nt_contribution_500 = nt_contribution_500.where((nt_contribution_500 >= 0)|(nt_contribution_500.isnull()), 0)

### Check Amite Transect line

In [0]:

amite_line_extended = gpd.read_file(f'{data_dir1}/amite_line_pass_manchac_centered_userpoints.json').to_crs('EPSG:26915')
ax = amite_line_extended.plot(figsize=(24, 12))
ctx.add_basemap(ax, crs=amite_line_extended.crs, source=ctx.providers.CartoDB.Positron)

## Load and Plot Domain Boundary

In [0]:
from rasterio import features
import shapely.geometry
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

# Extract non-NaN shapes from the raster
shapes = list(
    features.shapes(
        compound_10, 
        transform=compound_10.rio.transform(), 
        mask=~np.isnan(compound_10)
    )
)

# Convert shapes into geometries
geometries = []
values = []

for geom, value in shapes:
    geometries.append(shapely.geometry.shape(geom))
    values.append(value)

# Build GeoDataFrame with proper CRS
domain_gdf = gpd.GeoDataFrame(geometry=geometries, crs=compound_10.rio.crs)

# Dissolve to merge all polygons into a single shape
domain_gdf = domain_gdf.dissolve()

# Plot the boundary with formatting
fig, ax = plt.subplots(figsize=(8, 8))
domain_gdf.exterior.plot(ax=ax, edgecolor='blue')

# Add title and axis labels
ax.set_title("Compound_10 Domain Boundary", fontsize=14)
ax.set_xlabel("Easting (meters)", fontsize=12)
ax.set_ylabel("Northing (meters)", fontsize=12)
ax.ticklabel_format(useOffset=False, style='plain')

plt.tight_layout()
plt.show()


## Comparison to old Data from Purdue

In [0]:
combined_fig_10_1_rep = combined_fig_10_1.rio.reproject_match(combined_depth_10).rio.clip(domain_gdf.geometry, domain_gdf.crs)
combined_fig_50_1_rep = combined_fig_50_1.rio.reproject_match(combined_depth_50).rio.clip(domain_gdf.geometry, domain_gdf.crs)
combined_fig_100_1_rep = combined_fig_100_1.rio.reproject_match(combined_depth_100).rio.clip(domain_gdf.geometry, domain_gdf.crs)

combined_fig_10_rep = combined_fig_10.rio.reproject_match(combined_depth_10).rio.clip(domain_gdf.geometry, domain_gdf.crs)
combined_fig_50_rep = combined_fig_50.rio.reproject_match(combined_depth_50).rio.clip(domain_gdf.geometry, domain_gdf.crs)
combined_fig_100_rep = combined_fig_100.rio.reproject_match(combined_depth_100).rio.clip(domain_gdf.geometry, domain_gdf.crs)

In [0]:
fig, axes = plt.subplots(6, 3, figsize=(18, 36))

# First row of plots
(combined_fig_50_1_rep - combined_fig_10_1_rep).plot(ax=axes[0, 0], vmin=-5, vmax=5)
axes[0, 0].set_title('50_1_rep - 10_1_rep')

(combined_fig_100_1_rep - combined_fig_50_1_rep).plot(ax=axes[0, 1], vmin=-5, vmax=5)
axes[0, 1].set_title('100_1_rep - 50_1_rep')

(combined_fig_100_1_rep - combined_fig_10_1_rep).plot(ax=axes[0, 2], vmin=-5, vmax=5)
axes[0, 2].set_title('100_1_rep - 10_1_rep')

# Second row of histograms
(combined_fig_50_1_rep - combined_fig_10_1_rep).plot.hist(ax=axes[1, 0], bins=50, range=(-5, 5))
axes[1, 0].set_title('Histogram: 50_1_rep - 10_1_rep')

(combined_fig_100_1_rep - combined_fig_50_1_rep).plot.hist(ax=axes[1, 1], bins=50, range=(-5, 5))
axes[1, 1].set_title('Histogram: 100_1_rep - 50_1_rep')

(combined_fig_100_1_rep - combined_fig_10_1_rep).plot.hist(ax=axes[1, 2], bins=50, range=(-5, 5))
axes[1, 2].set_title('Histogram: 100_1_rep - 10_1_rep')

# Third row of plots
(combined_fig_50_rep - combined_fig_10_rep).plot(ax=axes[2, 0], vmin=-5, vmax=5)
axes[2, 0].set_title('50_rep - 10_rep')

(combined_fig_100_rep - combined_fig_50_rep).plot(ax=axes[2, 1], vmin=-5, vmax=5)
axes[2, 1].set_title('100_rep - 50_rep')

(combined_fig_100_rep - combined_fig_10_rep).plot(ax=axes[2, 2], vmin=-5, vmax=5)
axes[2, 2].set_title('100_rep - 10_rep')

# Fourth row of histograms
(combined_fig_50_rep - combined_fig_10_rep).plot.hist(ax=axes[3, 0], bins=50, range=(-5, 5))
axes[3, 0].set_title('Histogram: 50_rep - 10_rep')

(combined_fig_100_rep - combined_fig_50_rep).plot.hist(ax=axes[3, 1], bins=50, range=(-5, 5))
axes[3, 1].set_title('Histogram: 100_rep - 50_rep')

(combined_fig_100_rep - combined_fig_10_rep).plot.hist(ax=axes[3, 2], bins=50, range=(-5, 5))
axes[3, 2].set_title('Histogram: 100_rep - 10_rep')

# Fifth row of plots
(combined_50 - combined_10).plot(ax=axes[4, 0], vmin=-5, vmax=5)
axes[4, 0].set_title('combined_50 - combined_10')

(combined_100 - combined_50).plot(ax=axes[4, 1], vmin=-5, vmax=5)
axes[4, 1].set_title('combined_100 - combined_50')

(combined_100 - combined_10).plot(ax=axes[4, 2], vmin=-5, vmax=5)
axes[4, 2].set_title('combined_100 - combined_10')

# Sixth row of histograms
(combined_50 - combined_10).plot.hist(ax=axes[5, 0], bins=50, range=(-5, 5))
axes[5, 0].set_title('Histogram: combined_50 - combined_10')

(combined_100 - combined_50).plot.hist(ax=axes[5, 1], bins=50, range=(-5, 5))
axes[5, 1].set_title('Histogram: combined_100 - combined_50')

(combined_100 - combined_10).plot.hist(ax=axes[5, 2], bins=50, range=(-5, 5))
axes[5, 2].set_title('Histogram: combined_100 - combined_10')

plt.tight_layout()
plt.show()

In [0]:
#I couldn't figure out how to fix order issues with the amite line in arcpro, so we do it here
fixed_line = ensure_connected_linestring(amite_line.geometry[0])
fixed_line_gdf = gpd.GeoDataFrame({'geometry': [fixed_line]}, crs = 'EPSG:26915')


distances, values_10 = extract_raster_along_line(diff_10, fixed_line_gdf)
distances, values_50 = extract_raster_along_line(diff_50, fixed_line_gdf)
distances, values_100 = extract_raster_along_line(diff_100, fixed_line_gdf)
distances, values_500 = extract_raster_along_line(diff_500, fixed_line_gdf)

distances, compound_10_values = extract_raster_along_line(compound_10, fixed_line_gdf)
distances, compound_50_values = extract_raster_along_line(compound_50, fixed_line_gdf)
distances, compound_100_values = extract_raster_along_line(compound_100, fixed_line_gdf)
distances, compound_500_values = extract_raster_along_line(compound_500, fixed_line_gdf)

distances, pluvial_surge_riverine_10_values = extract_raster_along_line(pluvial_surge_riverine_10, fixed_line_gdf)
distances, pluvial_surge_riverine_50_values = extract_raster_along_line(pluvial_surge_riverine_50, fixed_line_gdf)
distances, pluvial_surge_riverine_100_values = extract_raster_along_line(pluvial_surge_riverine_100, fixed_line_gdf)
distances, pluvial_surge_riverine_500_values = extract_raster_along_line(pluvial_surge_riverine_500, fixed_line_gdf)



fig = plt.figure(figsize=(7.5, 12))  # academic width ~7.5 inches

gs = fig.add_gridspec(5, 2)
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[3, 0])
ax4 = fig.add_subplot(gs[4, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[3, 1])
ax8 = fig.add_subplot(gs[4, 1])
ax9 = fig.add_subplot(gs[0, :])  # overview map

# Plot lines - Left column: solid (Compound), dashed (Max of 3)
ax1.plot(distances/1000, compound_10_values, linestyle='-', color='black')
ax1.plot(distances/1000, pluvial_surge_riverine_10_values, linestyle='--', color='black')
ax2.plot(distances/1000, compound_50_values, linestyle='-', color='black')
ax2.plot(distances/1000, pluvial_surge_riverine_50_values, linestyle='--', color='black')
ax3.plot(distances/1000, compound_100_values, linestyle='-', color='black')
ax3.plot(distances/1000, pluvial_surge_riverine_100_values, linestyle='--', color='black')
ax4.plot(distances/1000, compound_500_values, linestyle='-', color='black')
ax4.plot(distances/1000, pluvial_surge_riverine_500_values, linestyle='--', color='black')

# Right column: Difference plots (black)
ax5.plot(distances/1000, values_10, color='black')
ax6.plot(distances/1000, values_50, color='black')
ax7.plot(distances/1000, values_100, color='black')
ax8.plot(distances/1000, values_500, color='black')

# Labels
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylabel('WSE (ft)', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)

for ax in [ax5, ax6, ax7, ax8]:
    ax.set_ylabel('WSE Difference (ft)', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)

# Titles
ax1.set_title('10-year', fontsize=11)
ax2.set_title('50-year', fontsize=11)
ax3.set_title('100-year', fontsize=11)
ax4.set_title('500-year', fontsize=11)
ax5.set_title('10-year', fontsize=11)
ax6.set_title('50-year', fontsize=11)
ax7.set_title('100-year', fontsize=11)
ax8.set_title('500-year', fontsize=11)

#fig.suptitle('Difference between compound flood depths and max of 3 individual drivers along Amite transect', fontsize=11)

# Top map
ax9.tick_params(axis='both', which='both', bottom=False, top=False,
                labelbottom=False, right=False, left=False, labelleft=False)
fixed_line_gdf.plot(ax=ax9)
domain_gdf.exterior.plot(ax=ax9)
ctx.add_basemap(ax9, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)

# Custom legend for left column
from matplotlib.lines import Line2D

fig.subplots_adjust(top=0.96, hspace=0.6, wspace=0.3)


# Custom line styles
legend_elements_left = [
    Line2D([0], [0], linestyle='-', color='black', label='Compounded Flood Elevation'),
    Line2D([0], [0], linestyle='--', color='black', label='Max of Pluvial, Fluvial, and Coastal Drivers')
]

from matplotlib.patches import Patch  # add this import at the top if not already present

legend_elements_right = [
    Patch(facecolor='none', edgecolor='none', label='Flood Depth Difference of Compound Flood\nand Max of Pluvial, Fluvial, and Coastal Drivers')
]


#Axes for placing column legends
ax_legend_left = fig.add_axes([0.07, 0.79, 0.42, 0.05])   # [left, bottom, width, height]
ax_legend_right = fig.add_axes([0.52, 0.79, 0.42, 0.05])


# Turn off everything for dummy axes
for ax in [ax_legend_left, ax_legend_right]:
    ax.axis('off')

# Left column legend
ax_legend_left.legend(
    handles=legend_elements_left,
    loc='center',
    fontsize=10,
    frameon=False
)

# Right column legend
ax_legend_right.legend(
    handles=legend_elements_right,
    loc='center',
    fontsize=10,
    frameon=False
)



# Figure 7
In both feet and meters 

In [0]:


fixed_line = ensure_connected_linestring(amite_line.geometry[0])
fixed_line_gdf = gpd.GeoDataFrame({'geometry': [fixed_line]}, crs = 'EPSG:26915')


distances, values_10 = extract_raster_along_line(diff_10, fixed_line_gdf)
distances, values_50 = extract_raster_along_line(diff_50, fixed_line_gdf)
distances, values_100 = extract_raster_along_line(diff_100, fixed_line_gdf)
distances, values_500 = extract_raster_along_line(diff_500, fixed_line_gdf)

distances, compound_10_values = extract_raster_along_line(compound_10, fixed_line_gdf)
distances, compound_50_values = extract_raster_along_line(compound_50, fixed_line_gdf)
distances, compound_100_values = extract_raster_along_line(compound_100, fixed_line_gdf)
distances, compound_500_values = extract_raster_along_line(compound_500, fixed_line_gdf)

distances, pluvial_surge_riverine_10_values = extract_raster_along_line(pluvial_surge_riverine_10, fixed_line_gdf)
distances, pluvial_surge_riverine_50_values = extract_raster_along_line(pluvial_surge_riverine_50, fixed_line_gdf)
distances, pluvial_surge_riverine_100_values = extract_raster_along_line(pluvial_surge_riverine_100, fixed_line_gdf)
distances, pluvial_surge_riverine_500_values = extract_raster_along_line(pluvial_surge_riverine_500, fixed_line_gdf)

# ---------- x in km ----------
x_km = distances / 1000.0
tick_labels = [0, 20, 40, 60, 80, 100, 120]

# ---------- figure + axes ----------
fig = plt.figure(figsize=(7.5, 12))
gs = fig.add_gridspec(5, 2)
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[3, 0])
ax4 = fig.add_subplot(gs[4, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[3, 1])
ax8 = fig.add_subplot(gs[4, 1])
#ax9 = fig.add_subplot(gs[0, :])

# ---------- left curves ----------
ax1.plot(x_km, compound_10_values,  '-',  color='black')
ax1.plot(x_km, pluvial_surge_riverine_10_values,  '--', color='black')

ax2.plot(x_km, compound_50_values,  '-',  color='black')
ax2.plot(x_km, pluvial_surge_riverine_50_values,  '--', color='black')

ax3.plot(x_km, compound_100_values, '-',  color='black')
ax3.plot(x_km, pluvial_surge_riverine_100_values, '--', color='black')

ax4.plot(x_km, compound_500_values, '-',  color='black')
ax4.plot(x_km, pluvial_surge_riverine_500_values, '--', color='black')

# ---------- right curves ----------
ax5.plot(x_km, values_10,   color='black')
ax6.plot(x_km, values_50,   color='black')
ax7.plot(x_km, values_100,  color='black')
ax8.plot(x_km, values_500,  color='black')

# ---------- CFTZ (rows 1–2 'between'; rows 3–4 'to_right') ----------
rows = [
    (ax1, ax5, values_10,  "between"),   # 10-year
    (ax2, ax6, values_50,  "between"),   # 50-year
    (ax3, ax7, values_100, "between"),  # 100-year
    (ax4, ax8, values_500, "between"),  # 500-year
]

CROSSING_LEVEL =.33
for L_ax, R_ax, diff_vals, mode in rows:
    xs = all_level_crossings(x_km, diff_vals, level=CROSSING_LEVEL )
    xs= group_and_average(xs)
    print(xs)
    # choose outermost pair; if less than 2 crossings, fall back to a tiny span
    if len(xs) <= 2:
        xL, xR = xs[0], xs[-1]
    elif len(xs) == 1:
        xL, xR = xs[0], xs[0] + max(0.01, 0.01 * (x_km[-1] - x_km[0]))  # small visible span
    elif len(xs) > 2:
        xL, xL1, xR, xR1 = xs[0], xs[-1], xs[1], xs[-2]
    else:
        # if no crossing, skip verticals/dotted line
        continue
    if len(xs) <= 2:
        draw_cftz(L_ax, R_ax, xL, xR, mode=mode, label="CFTZ")
    else:
        draw_cftz(L_ax, R_ax, xL, xR, mode=mode, label="CFTZ")
        draw_cftz(L_ax, R_ax, xL1, xR1, mode=mode, label="CFTZ")

# ---------- labels/ticks ----------

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylabel('WSE, ft NAVD88', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)
    ax.set_xticks(tick_labels)
    ax.set_xticklabels([str(t) for t in tick_labels])
    set_integer_yticks(ax)

for ax in [ax5, ax6, ax7, ax8]:
    ax.set_ylabel('WSE Difference, ft', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)
    ax.set_xticks(tick_labels)
    ax.set_xticklabels([str(t) for t in tick_labels])
    set_integer_yticks(ax)

# ---------- titles ----------
ax1.set_title('10-year',  fontsize=11); ax5.set_title('10-year',  fontsize=11)
ax2.set_title('50-year',  fontsize=11); ax6.set_title('50-year',  fontsize=11)
ax3.set_title('100-year', fontsize=11); ax7.set_title('100-year', fontsize=11)
ax4.set_title('500-year', fontsize=11); ax8.set_title('500-year', fontsize=11)

# ---------- overview map ----------

#ax9.tick_params(axis='both', which='both', bottom=False, top=False,
#                labelbottom=False, right=False, left=False, labelleft=False)
#fixed_line_gdf.plot(ax=ax9)
#domain_gdf.exterior.plot(ax=ax9)
#ctx.add_basemap(ax9, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)

# ---------- legends ----------
fig.subplots_adjust(top=0.96, hspace=0.6, wspace=0.35)
legend_elements_left = [
    Line2D([0], [0], linestyle='-',  color='black', label='Compounded Flood Elevation'),
    Line2D([0], [0], linestyle='--', color='black', label='Max of Pluvial, Fluvial, and Coastal Floods')
]
legend_elements_right = [
    Patch(facecolor='none', edgecolor='none')
          #label='WSE Difference')
]
ax_legend_left  = fig.add_axes([0.07, 0.02, 0.42, 0.05])
ax_legend_right = fig.add_axes([0.52, 0.79, 0.42, 0.05])
for ax in (ax_legend_left, ax_legend_right):
    ax.axis('off')
ax_legend_left.legend(handles=legend_elements_left,  loc='lower center', fontsize=10, frameon=False)
ax_legend_right.legend(handles=legend_elements_right, loc='center', fontsize=10, frameon=False)

#fig.legend(handles=legend_elements_left, loc='lower center', fontsize=11, frameon=False, ncol=2, bbox_to_anchor=(0.5, 0.03))

plt.savefig("transects.pdf", bbox_inches="tight")

In [0]:

# ---------- x in km ----------
x_km = distances / 1000.0
tick_labels = [0, 20, 40, 60, 80, 100, 120]

# ---------- figure + axes ----------
fig = plt.figure(figsize=(7.5, 12))
gs = fig.add_gridspec(5, 2)
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[3, 0])
ax4 = fig.add_subplot(gs[4, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[3, 1])
ax8 = fig.add_subplot(gs[4, 1])
#ax9 = fig.add_subplot(gs[0, :])

# ---------- left curves ----------
ax1.plot(x_km, compound_10_values*0.3048,  '-',  color='black')
ax1.plot(x_km, pluvial_surge_riverine_10_values*0.3048,  '--', color='black')

ax2.plot(x_km, compound_50_values*0.3048,  '-',  color='black')
ax2.plot(x_km, pluvial_surge_riverine_50_values*0.3048,  '--', color='black')

ax3.plot(x_km, compound_100_values*0.3048, '-',  color='black')
ax3.plot(x_km, pluvial_surge_riverine_100_values*0.3048, '--', color='black')

ax4.plot(x_km, compound_500_values*0.3048, '-',  color='black')
ax4.plot(x_km, pluvial_surge_riverine_500_values*0.3048, '--', color='black')

# ---------- right curves ----------
ax5.plot(x_km, values_10*0.3048,   color='black')
ax6.plot(x_km, values_50*0.3048,   color='black')
ax7.plot(x_km, values_100*0.3048,  color='black')
ax8.plot(x_km, values_500*0.3048,  color='black')

# ---------- CFTZ (rows 1–2 'between'; rows 3–4 'to_right') ----------
rows = [
    (ax1, ax5, values_10*0.3048,  "between"),   # 10-year
    (ax2, ax6, values_50*0.3048,  "between"),   # 50-year
    (ax3, ax7, values_100*0.3048, "between"),  # 100-year
    (ax4, ax8, values_500*0.3048, "between"),  # 500-year
]

CROSSING_LEVEL =.33*0.3048
for L_ax, R_ax, diff_vals, mode in rows:
    xs = all_level_crossings(x_km, diff_vals, level=CROSSING_LEVEL )
    xs= group_and_average(xs)
    print(xs)
    # choose outermost pair; if less than 2 crossings, fall back to a tiny span
    if len(xs) <= 2:
        xL, xR = xs[0], xs[-1]
    elif len(xs) == 1:
        xL, xR = xs[0], xs[0] + max(0.01, 0.01 * (x_km[-1] - x_km[0]))  # small visible span
    elif len(xs) > 2:
        xL, xL1, xR, xR1 = xs[0], xs[-1], xs[1], xs[-2]
    else:
        # if no crossing, skip verticals/dotted line
        continue
    if len(xs) <= 2:
        draw_cftz(L_ax, R_ax, xL, xR, mode=mode, label="CFTZ")
    else:
        draw_cftz(L_ax, R_ax, xL, xR, mode=mode, label="CFTZ")
        draw_cftz(L_ax, R_ax, xL1, xR1, mode=mode, label="CFTZ")

# ---------- labels/ticks ----------

for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylabel('WSE, m NAVD88', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)
    ax.set_xticks(tick_labels)
    ax.set_xticklabels([str(t) for t in tick_labels])
    #set_integer_yticks(ax)

for ax in [ax5, ax6, ax7, ax8]:
    ax.set_ylabel('WSE Difference, m', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)
    ax.set_xticks(tick_labels)
    ax.set_xticklabels([str(t) for t in tick_labels])
    #set_integer_yticks(ax)

# ---------- titles ----------
ax1.set_title('10-year',  fontsize=11); ax5.set_title('10-year',  fontsize=11)
ax2.set_title('50-year',  fontsize=11); ax6.set_title('50-year',  fontsize=11)
ax3.set_title('100-year', fontsize=11); ax7.set_title('100-year', fontsize=11)
ax4.set_title('500-year', fontsize=11); ax8.set_title('500-year', fontsize=11)

# ---------- overview map ----------
#ax9.tick_params(axis='both', which='both', bottom=False, top=False,
#                labelbottom=False, right=False, left=False, labelleft=False)
#fixed_line_gdf.plot(ax=ax9)
#domain_gdf.exterior.plot(ax=ax9)
#ctx.add_basemap(ax9, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)

# ---------- legends ----------
fig.subplots_adjust(top=0.96, hspace=0.6, wspace=0.3)
legend_elements_left = [
    Line2D([0], [0], linestyle='-',  color='black', label='Compounded Flood Elevation'),
    Line2D([0], [0], linestyle='--', color='black', label='Max of Pluvial, Fluvial, and Coastal Floods')
]
legend_elements_right = [
    Patch(facecolor='none', edgecolor='none')
          #label='WSE Difference')
]
ax_legend_left  = fig.add_axes([0.07, 0.02, 0.42, 0.05])
ax_legend_right = fig.add_axes([0.52, 0.79, 0.42, 0.05])
for ax in (ax_legend_left, ax_legend_right):
    ax.axis('off')
ax_legend_left.legend(handles=legend_elements_left,  loc='lower center', fontsize=10, frameon=False)
ax_legend_right.legend(handles=legend_elements_right, loc='center', fontsize=10, frameon=False)

#fig.legend(handles=legend_elements_left, loc='lower center', fontsize=11, frameon=False, ncol=2, bbox_to_anchor=(0.5, 0.03))

plt.savefig("transects.pdf", bbox_inches="tight")

# Transition Zone area for the Paper

In [0]:
diff10_area = diff_10 > difference_threshold
diff50_area_increase_only = (diff_50 > difference_threshold)|(diff_10 > difference_threshold)
diff100_area_increase_only = (diff_100 > difference_threshold)|(diff_50 > difference_threshold)|(diff_10 > difference_threshold)
diff500_area_increase_only = (diff_500 > difference_threshold)|(diff_100 > difference_threshold)|(diff_50 > difference_threshold)|(diff_10 > difference_threshold)

diff10_area = diff_10 > difference_threshold
diff50_area = (diff_50 > difference_threshold)
diff100_area = (diff_100 > difference_threshold)
diff500_area = (diff_500 > difference_threshold)

#one of the dimensions is negative for unclear reasons, probably to do with the ordering of cells
cell_area = np.abs(diff10_area.rio.resolution()[0] * diff10_area.rio.resolution()[1])

diff10_area_val = (diff10_area.sum() * cell_area).values/10**6
diff50_area_val = (diff50_area.sum() * cell_area).values/10**6
diff100_area_val = (diff100_area.sum() * cell_area).values/10**6
diff500_area_val = (diff500_area.sum() * cell_area).values/10**6


diff10_area_increasing_val = (diff10_area.sum() * cell_area).values/10**6
diff50_area_increasing_val = (diff50_area_increase_only.sum() * cell_area).values/10**6
diff100_area_increasing_val = (diff100_area_increase_only.sum() * cell_area).values/10**6
diff500_area_increasing_val = (diff500_area_increase_only.sum() * cell_area).values/10**6


print(f'the area of the transition zone at return periods of 10, 50, 100, and 500 years are {diff10_area_val}, {diff50_area_val}, {diff100_area_val}, and {diff500_area_val} square km respectively')

print(f'the area considering the area of the TZ only increses with RP of the 10, 50, 100, and 500 years are {diff10_area_increasing_val}, {diff50_area_increasing_val}, {diff100_area_increasing_val}, and {diff500_area_increasing_val} square km respectively')

# Figure 6
In both feet and meters

In [0]:


# ---------------- settings ----------------
# Convert from feet to meters (multiply by 0.3048)
LOW_CUT = 0.33 * 0.3048    # treat near-zero background as transparent (but don't create gaps)
HIGH_DROP = 3.5 * 0.3048    # drop the yellow-line outliers
VMIN, VMAX = 0, 2.25 * 0.3048
RASTER_CRS = "EPSG:26915"

# colormap + normalizer
blues = plt.cm.Blues

cmap = LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

# make values below LOW_CUT fully transparent (for pretty basemap)
cmap.set_under((0, 0, 0, 0))
norm = Normalize(vmin=LOW_CUT, vmax=VMAX)

def prep(da):
    # Convert to meters and keep finite values, drop the artifact band, and make near-zeros transparent
    da_m = da * 0.3048  # convert feet to meters
    da2 = da_m.where(np.isfinite(da_m) & (da_m < HIGH_DROP))
    da2 = da2.where(da2 >= LOW_CUT)  # below vmin -> transparent because of set_under
    return da2

fig, ax = plt.subplots(2, 2, figsize=(9.6, 7.6), dpi=120)

# -------- plot panels (no gaps, no yellow band) --------
prep(diff_10 ).plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_50 ).plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_100).plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_500).plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# -------- basemap + cosmetics --------
for a, title, area_val in zip(ax.flat, ["10-year", "50-year", "100-year", "500-year"],
                              [diff10_area_val, diff50_area_val, diff100_area_val, diff500_area_val]):
    ctx.add_basemap(a, crs=RASTER_CRS, source=ctx.providers.CartoDB.Positron)
    a.set_aspect("equal")
    a.set_title(title, fontsize=12, pad=6)
    a.set_xlabel(""); a.set_ylabel("")
    a.tick_params(axis='both', which='both',
                  bottom=False, top=False, right=False, left=False,
                  labelbottom=False, labelleft=False)
    a.text(0.05, 0.05, f"CFTZ Area:\n{round(area_val)} km²", transform=a.transAxes, 
           fontsize=8, color='black', ha='left')

# overlay the Amite transect with 0/20/40/60/80 km marks on EACH panel
for a in ax.flat:
    overlay_transect(a, fixed_line, km_ticks=(0,20,40,60,80,100,115),
                     line_lw=.5, dot_size=15, label_fontsize=7,
                     ensure_north_start=False, label_position='right')

# -------- shared colorbar --------
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.70])   # [left, bottom, width, height]
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Difference in WSE, m", fontsize=12)

# Create ticks rounded to nearest tenth
ticks = np.linspace(LOW_CUT, VMAX, 9)
cb.set_ticks(ticks)
cb.set_ticklabels([f"{t:.1f}" for t in ticks])  # Format to 1 decimal place
cb.ax.tick_params(labelsize=10)
cb.outline.set_visible(False)

plt.subplots_adjust(left=0.05, right=0.89, top=0.92, bottom=0.08, wspace=0.08, hspace=0.10)
plt.savefig("transition_panels.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [0]:

# ---------------- settings ----------------
LOW_CUT = 0.33    # treat near-zero background as transparent (but don't create gaps)
HIGH_DROP = 3.5    # drop the yellow-line outliers
VMIN, VMAX = 0, 2.25
RASTER_CRS = "EPSG:26915"

# colormap + normalizer
blues = plt.cm.Blues

cmap = LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

# make values below LOW_CUT fully transparent (for pretty basemap)
cmap.set_under((0, 0, 0, 0))
norm = Normalize(vmin=LOW_CUT, vmax=VMAX)

def prep(da):
    # keep finite values, drop the artifact band, and make near-zeros transparent
    da2 = da.where(np.isfinite(da) & (da < HIGH_DROP))
    da2 = da2.where(da2 >= LOW_CUT)  # below vmin -> transparent because of set_under
    return da2

fig, ax = plt.subplots(2, 2, figsize=(9.6, 7.6), dpi=120)

# -------- plot panels (no gaps, no yellow band) --------
prep(diff_10 ).plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_50 ).plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_100).plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
prep(diff_500).plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# -------- basemap + cosmetics --------
for a, title, area_val in zip(ax.flat, ["10-year", "50-year", "100-year", "500-year"],
                              [diff10_area_val, diff50_area_val, diff100_area_val, diff500_area_val]):
    ctx.add_basemap(a, crs=RASTER_CRS, source=ctx.providers.CartoDB.Positron)
    a.set_aspect("equal")
    a.set_title(title, fontsize=12, pad=6)
    a.set_xlabel(""); a.set_ylabel("")
    a.tick_params(axis='both', which='both',
                  bottom=False, top=False, right=False, left=False,
                  labelbottom=False, labelleft=False)
    a.text(0.05, 0.05, f"CFTZ Area:\n{round(area_val)} km²", transform=a.transAxes, 
           fontsize=8, color='black', ha='left')

# overlay the Amite transect with 0/20/40/60/80 km marks on EACH panel
for a in ax.flat:
    overlay_transect(a, fixed_line, km_ticks=(0,20,40,60,80,100,115),
                     line_lw=.5, dot_size=15, label_fontsize=7,
                     ensure_north_start=False, label_position='right')

# -------- shared colorbar --------
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.70])   # [left, bottom, width, height]
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Difference in WSE (ft)", fontsize=12)
ticks = np.linspace(LOW_CUT, VMAX, 9)
cb.set_ticks(ticks)
cb.set_ticklabels([f"{t:g}" for t in ticks])
cb.ax.tick_params(labelsize=10)
cb.outline.set_visible(False)

plt.subplots_adjust(left=0.05, right=0.89, top=0.92, bottom=0.08, wspace=0.08, hspace=0.10)
plt.savefig("transition_panels.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [0]:
arr = (prep(diff_500)/pluvial_surge_riverine_50*100)
print(np.nanmin(arr.values), np.nanmax(arr.values))

In [0]:
arr = (prep(diff_500)/pluvial_surge_riverine_500*100)
print(np.nanmin(arr.values), np.nanmax(arr.values))

# Figure 8

In [0]:
# ---------------- settings ----------------
LOW_CUT = 0.33     # treat near-zero background as transparent (but don't create gaps)
HIGH_DROP = 3.5    # drop the yellow-line outliers
VMIN, VMAX = 0, 2.25
RASTER_CRS = "EPSG:26915"

# colormap + normalizer
cmap = cm.get_cmap("cividis").copy()

# make values below LOW_CUT fully transparent (for pretty basemap)
cmap.set_under((0, 0, 0, 0))
norm = Normalize(vmin=0, vmax=50)

def prep(da):
    # keep finite values, drop the artifact band, and make near-zeros transparent
    da2 = da.where(np.isfinite(da) & (da < HIGH_DROP))
    da2 = da2.where(da2 >= LOW_CUT)  # below vmin -> transparent because of set_under
    return da2

fig, ax = plt.subplots(2, 2, figsize=(9.6, 7.6), dpi=120)

# -------- plot panels (no gaps, no yellow band) --------
(prep(diff_10 )/pluvial_surge_riverine_10*100).plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(prep(diff_50 )/pluvial_surge_riverine_50*100).plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(prep(diff_100)/pluvial_surge_riverine_100*100).plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(prep(diff_500)/pluvial_surge_riverine_500*100).plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# -------- basemap + cosmetics --------
for a, title in zip(ax.flat, ["10-year","50-year","100-year","500-year"]):
    ctx.add_basemap(a, crs=RASTER_CRS, source=ctx.providers.CartoDB.Positron)
    a.set_aspect("equal")
    a.set_title(title, fontsize=12, pad=6)
    a.set_xlabel(""); a.set_ylabel("")
    a.tick_params(axis='both', which='both',
                  bottom=False, top=False, right=False, left=False,
                  labelbottom=False, labelleft=False)

# overlay the Amite transect with 0/20/40/60/80 km marks on EACH panel
# (set ensure_north_start=True if you want "0" forced to the top/north)
for a in ax.flat:
    overlay_transect(a, fixed_line, km_ticks=(0,20,40,60,80,100,115),
                     line_lw=.75, dot_size=15, label_fontsize=7,
                     ensure_north_start=False, label_position='right')

# -------- shared colorbar --------
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.70])   # [left, bottom, width, height]
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Percent increase", fontsize=12)
# nice ticks starting at 0 even though vmin is LOW_CUT
ticks = np.linspace(0, 50, 5)
cb.set_ticks(ticks)
cb.set_ticklabels([f"{t:g}" for t in ticks])
cb.ax.tick_params(labelsize=10)
cb.outline.set_visible(False)

plt.subplots_adjust(left=0.05, right=0.89, top=0.92, bottom=0.08, wspace=0.08, hspace=0.10)
plt.savefig("transition_percent_panels.pdf", dpi=300, bbox_inches="tight")
plt.show()

## Create Outlines of CFTZs

In [0]:
# --- cumulative 0.5–4.0 ft transition outlines (outliers removed) + QA plot ---

#I have to reimport the shapely Polygon functionality b/c it conflicts with "from matplotlib.patches import Patch, Polygon"
from shapely.geometry import LineString, Point, MultiLineString, Polygon
# ---- build compounded masks with outlier cap ----
m10  = make_band_mask(diff_10,  THRESH_MIN, THRESH_MAX)
mc50  = compound_masks(m10,  make_band_mask(diff_50,  THRESH_MIN, THRESH_MAX))
mc100 = compound_masks(mc50,  make_band_mask(diff_100, THRESH_MIN, THRESH_MAX))
mc500 = compound_masks(mc100, make_band_mask(diff_500, THRESH_MIN, THRESH_MAX))
m50  = make_band_mask(diff_50,  THRESH_MIN, THRESH_MAX)
m100 = make_band_mask(diff_100,  THRESH_MIN, THRESH_MAX)
m500 = make_band_mask(diff_500,  THRESH_MIN, THRESH_MAX)

# ---- outlines ----
g10  = mask_to_outline_gdf(m10,  diff_10)
g50  = mask_to_outline_gdf(m50,  diff_50)
g100 = mask_to_outline_gdf(m100, diff_100)
g500 = mask_to_outline_gdf(m500, diff_500)

gc = mask_to_outline_gdf(mc500, diff_500)

print("10-yr outline empty?",  g10.empty)
print("50-yr outline empty?",  g50.empty)
print("100-yr outline empty?", g100.empty)
print("500-yr outline empty?", g500.empty)

# ---- save for reuse ----
save_geojson(g10,  "../data/CFTZ_outline_10_yr.geojson")
save_geojson(g50,  "../data/CFTZ_outline_50_yr.geojson")
save_geojson(g100, "../data/CFTZ_outline_100_yr.geojson")
save_geojson(g500, "../data/CFTZ_outline_500_yr.geojson")

# ---- quick QA plot ----
def qa_outline(ax, g, title):
    if g.empty:
        ax.set_title(title)
        ax.text(0.5, 0.5, "EMPTY", ha="center", va="center", transform=ax.transAxes)
        ax.set_axis_off()
        return
    g3857 = g.to_crs(3857)
    minx, miny, maxx, maxy = g3857.total_bounds
    pad_x, pad_y = (maxx - minx)*0.05, (maxy - miny)*0.05
    ax.set_xlim(minx - pad_x, maxx + pad_x); ax.set_ylim(miny - pad_y, maxy + pad_y)
    ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)  # Web Mercator
    g3857.plot(ax=ax, color="royalblue", linewidth=2.0, zorder=5)
    ax.set_aspect("equal")
    ax.set_title(title, fontsize=13)
    ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)

fig, axes = plt.subplots(2, 2, figsize=(11, 7.2))
qa_outline(axes[0,0], g10,  "10-year")
qa_outline(axes[0,1], g50,  "50-year")
qa_outline(axes[1,0], g100, "100-year")
qa_outline(axes[1,1], g500, "500-year")
plt.tight_layout()
plt.show()


### Load Bilskie Paper Comparison Polygon

In [0]:
bilskie_path = "../data/flood_zones_FeaturesToJSON.geojson"
zones = gpd.read_file(bilskie_path)

# match CRS with rasters
if zones.crs is None:
    zones.set_crs(3857, inplace=True)
elif zones.crs != 3857:
    zones = zones.to_crs(3857)

transition_gdf = get_transition_polys(zones)


transition_gdf_simplified = simplify_transition_gdf(transition_gdf)

transition_gdf_simplified26915=transition_gdf_simplified.to_crs(26915)

### Calculate Areas of CFTZs

In [0]:


# If the lines form a closed polygon
merged_lines = linemerge(gc.to_crs(26915).geometry.tolist())
polygons = list(polygonize([merged_lines]))
if polygons:
    total_area_km2 = sum(poly.area for poly in polygons) / 1e6
    print(f"Total polygon area: {total_area_km2:.2f} km²")

# If the lines form a closed polygon
merged_lines = linemerge(transition_gdf_simplified.to_crs(26915).geometry.tolist())
polygons = list(polygonize([merged_lines]))
if polygons:
    total_area_km2 = sum(poly.area for poly in polygons) / 1e6
    print(f"Total polygon area: {total_area_km2:.2f} km²")
    

In [0]:


# Collect all LineStrings from all rows
all_lines = []

for geom in transition_gdf_simplified.to_crs(26915).geometry:
    if isinstance(geom, MultiLineString):
        # Extract individual LineStrings from MultiLineString
        all_lines.extend(list(geom.geoms))
    else:
        # Single LineString case
        all_lines.append(geom)

# Now merge and polygonize all lines
merged_lines = linemerge(all_lines)
polygons = list(polygonize([merged_lines]))
if polygons:
    total_area_km2 = sum(poly.area for poly in polygons) / 1e6
    print(f"Total polygon area: {total_area_km2:.2f} km²")

In [0]:


# Collect all LineStrings from all rows
all_lines = []

for geom in g50.to_crs(26915).geometry:
    if isinstance(geom, MultiLineString):
        # Extract individual LineStrings from MultiLineString
        all_lines.extend(list(geom.geoms))
    else:
        # Single LineString case
        all_lines.append(geom)

# Now merge and polygonize all lines
merged_lines = linemerge(all_lines)
polygons = list(polygonize([merged_lines]))
if polygons:
    total_area_km2 = sum(poly.area for poly in polygons) / 1e6
    print(f"Total polygon area: {total_area_km2:.2f} km²")

In [0]:
g10.plot()


# Figure 13

In [0]:


outlines = [g500, g100, g50, g10]
labels = ["500-year", "100-year", "50-year", "10-year"]

colors = plt.cm.Greys(np.linspace(0.2, 0.95, len(outlines)))
linestyles = ["solid", "solid", "solid", "solid"]

fig, ax = plt.subplots(figsize=(10, 8))

for g, label, ls, color in zip(outlines, labels, linestyles, colors):
    if not g.empty:
        g3857 = g.to_crs(3857)
        g3857.plot(ax=ax,
                   edgecolor='white',
                   facecolor=color,
                   alpha=1,
                   linewidth=.2,
                   linestyle=ls,
                   zorder=5)
g3857 = gc.to_crs(3857)
g3857.plot(ax=ax,
           color='black',
           alpha=1,
           linewidth=1,
           zorder=5)

if not transition_gdf_simplified.empty:
    transition_gdf_3857 = transition_gdf_simplified.to_crs(3857)
    transition_gdf_3857.plot(
        ax=ax,
        facecolor='none',
        edgecolor='white',
        linewidth=3.5,
        linestyle='solid',
        zorder=9,
        hatch='//'
    )
    transition_gdf_3857.plot(
        ax=ax,
        facecolor='none',
        edgecolor='black',
        linewidth=1.5,
        linestyle='dashed',
        zorder=10
    )

valid_outlines = [g for g in outlines if not g.empty]
if valid_outlines:
    all_g = gpd.GeoDataFrame(pd.concat(valid_outlines, ignore_index=True), crs=valid_outlines[0].crs).to_crs(3857)
    minx, miny, maxx, maxy = all_g.total_bounds
    pad_x, pad_y = (maxx - minx)*0.05, (maxy - miny)*0.05
    ax.set_xlim(minx - pad_x, maxx + pad_x)
    ax.set_ylim(miny - pad_y, maxy + pad_y)

ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron)
ax.set_aspect("equal")


###############
class HandlerOverlay(HandlerBase):
    """Custom handler that overlays multiple patches in one legend box."""
    def create_artists(self, legend, orig_handle,
                       xdescent, ydescent, width, height, fontsize, trans):
        artists = []
        for h in orig_handle:
            a = mpatches.Rectangle(
                (xdescent, ydescent), width, height,
                facecolor=h.get_facecolor(),
                edgecolor=h.get_edgecolor(),
                hatch=h.get_hatch(),
                linestyle=h.get_linestyle(),
                linewidth=h.get_linewidth(),
                transform=trans
            )
            artists.append(a)
        return artists

legend_elements = [
    Line2D([0], [0], color='black', lw=1, linestyle='solid', label='Overall')
]

legend_elements += [
    mpatches.Patch(facecolor=color, edgecolor='none', label=label)
    for label, color in zip(reversed(labels), reversed(colors))
]

# Define the two layers of the special patch
patch_hatch = mpatches.Patch(facecolor='lightgray', edgecolor='white', hatch='//')
patch_outline = mpatches.Patch(facecolor='none', edgecolor='black',
                               linestyle='--', linewidth=1.5)

# Add them as a tuple (we’ll overlay them in one box)
legend_elements.append((patch_hatch, patch_outline))

# Labels
labels_final = [
    h.get_label() if not isinstance(h, tuple) else "Bilskie & Hagen 2018"
    for h in legend_elements
]

ax.legend(
    handles=legend_elements,
    labels=labels_final,
    handler_map={tuple: HandlerOverlay()},  # overlay instead of side-by-side
    loc="lower right",
    bbox_to_anchor=(1.01, .01),
    title="CFTZ boundaries"
)
###########
ax.text(0.22, 0.91, "Hydrologic Flood Zone", transform=ax.transAxes,
        fontsize=13, fontweight='bold', va='top', ha='left', color='navy', zorder=20, rotation=20)
ax.text(0.85, 0.43, "Coastal Flood Zone", transform=ax.transAxes,
        fontsize=13, fontweight='bold', va='center', ha='right', color='darkred', zorder=20, rotation=50)

# Add test labels
ax.text(0.24, 0.43, "10-yr", transform=ax.transAxes, fontsize=8, color='white',
                zorder=12,
                path_effects=[__import__('matplotlib.patheffects').patheffects.withStroke(
                    linewidth=2, foreground='black')])
ax.text(0.195, 0.415, "50-yr", transform=ax.transAxes, fontsize=8, color='white',
                zorder=12,
                path_effects=[__import__('matplotlib.patheffects').patheffects.withStroke(
                    linewidth=2, foreground='black')])
ax.text(0.1395, 0.397, "100-yr", transform=ax.transAxes, fontsize=8, color='white',
                zorder=12,
                path_effects=[__import__('matplotlib.patheffects').patheffects.withStroke(
                    linewidth=2, foreground='black')])
ax.text(0.17, 0.48, "500-yr", transform=ax.transAxes, fontsize=8, color='white',
                zorder=12,
                path_effects=[__import__('matplotlib.patheffects').patheffects.withStroke(
                    linewidth=2, foreground='black')])

ax.annotate('', 
            xy=(0.04, 0.12),
            xytext=(0.04, 0.04),
            xycoords='axes fraction',
            textcoords='axes fraction',
            arrowprops=dict(
                facecolor='k', 
                edgecolor='k',
                arrowstyle='-|>',
                lw=3,
                mutation_scale=20
            ))
ax.text(0.04, 0.15, 'N', transform=ax.transAxes, 
        ha='center', va='center', fontsize=16, fontweight='bold', color='k')

def add_scalebar(ax, length_km, location=(0.08, 0.08), linewidth=4, text_offset=0.01):
    xlim = ax.get_xlim()
    ylim = ax.get_ylim()
    bar_length_m = length_km * 1000
    x_start = xlim[0] + (xlim[1] - xlim[0]) * location[0]
    y_start = ylim[0] + (ylim[1] - ylim[0]) * location[1]
    x_end = x_start + bar_length_m
    ax.plot([x_start, x_end], [y_start, y_start], color='k', lw=linewidth, solid_capstyle='butt', zorder=20)
    ax.text((x_start + x_end) / 2, y_start - (ylim[1] - ylim[0]) * text_offset,
            f"{length_km} km", ha='center', va='top', fontsize=12, color='k', zorder=21)

add_scalebar(ax, length_km=5, location=(0.08, 0.08), linewidth=4, text_offset=0.015)

ax.tick_params(bottom=False, left=False, labelbottom=False, labelleft=False)
plt.savefig("transition_compare.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [0]:
from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase

fig, ax = plt.subplots(2, 2, figsize=(9, 7.2))

# Flatten and filter real values for adaptive color range
all_vals = np.concatenate([
    diff_10_pluv.values.flatten(),
    diff_50_pluv.values.flatten(),
    diff_100_pluv.values.flatten(),
    diff_500_pluv.values.flatten()
])
all_vals = all_vals[~np.isnan(all_vals) & (all_vals > 0)]
max_val = np.percentile(all_vals, 99.5)  # true adaptive limit

norm = Normalize(vmin=0, vmax=25)
cmap = cm.get_cmap('plasma')

# Plot panels with shared color scale
img1 = diff_10_pluv.where(diff_10_pluv > difference_threshold).plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False)
img2 = diff_50_pluv.where(diff_50_pluv > difference_threshold).plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False)
img3 = diff_100_pluv.where(diff_100_pluv > difference_threshold).plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False)
img4 = diff_500_pluv.where(diff_500_pluv > difference_threshold).plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False)

# Basemaps and layout
for a in ax.flat:
    ctx.add_basemap(a, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)
    a.set_aspect('equal')
    a.set_xlabel('')
    a.set_ylabel('')
    a.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False, right=False, left=False, labelleft=False)

# Titles
ax[0,0].set_title('10-year', fontsize=11)
ax[0,1].set_title('50-year', fontsize=11)
ax[1,0].set_title('100-year', fontsize=11)
ax[1,1].set_title('500-year', fontsize=11)

# Shared vertical colorbar
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.7])
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Difference in WSE (ft)", fontsize=11)
cb.ax.tick_params(labelsize=11)
cb.outline.set_visible(False)

# Main title
fig.suptitle('Max(Riverine, Coastal, Pluvial) - Max(Riverine, Coastal)', fontsize=11, y=0.965)

# Layout spacing
plt.subplots_adjust(left=0.05, right=0.89, top=0.93, bottom=0.08, wspace=0.1, hspace=0.05)
plt.show()


# Figure 5
In both feet and meters


In [0]:

fig, ax = plt.subplots(2, 2, figsize=(9, 7.2))

# Fixed color scale range - converted to meters
vmin = 0
vmax = 12 * 0.3048  # Convert 12 feet to meters (~3.66 m)
norm = Normalize(vmin=vmin, vmax=vmax)

blues = plt.cm.Blues
cmap = LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

# Convert data to meters and filter
cd10 = (combined_depth_10 * 0.3048).where(combined_depth_10 > 0)
cd50 = (combined_depth_50 * 0.3048).where(combined_depth_50 > 0)
cd100 = (combined_depth_100 * 0.3048).where(combined_depth_100 > 0)
cd500 = (combined_depth_500 * 0.3048).where(combined_depth_500 > 0)

# Plot all compound flood depths
cd10.plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd50.plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd100.plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd500.plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# Basemaps and styling
for a in ax.flat:
    ctx.add_basemap(a, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)
    a.set_aspect('equal')
    a.set_xlabel('')
    a.set_ylabel('')
    a.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False, right=False, left=False, labelleft=False)

# Titles
titles = ['10-year', '50-year', '100-year', '500-year']
for i, title in enumerate(titles):
    row, col = divmod(i, 2)
    ax[row, col].set_title(title, fontsize=11)

# Shared vertical colorbar
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.7])
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Flood Depths, m", fontsize=11)
cb.ax.tick_params(labelsize=11)
cb.outline.set_visible(False)

# Set explicit tick positions and labels in meters
tick_positions = [0, 0.6096, 1.2192, 1.8288, 2.4384, 3.048, vmax]
cb.set_ticks(tick_positions)
cb.ax.set_yticklabels([
    f"{0:.1f}",
    f"{0.6:.1f}",
    f"{1.2:.1f}",
    f"{1.8:.1f}",
    f"{2.4:.1f}",
    f"{3.0:.1f}",
    f"$\geq${vmax:.1f}"
])

# Layout
plt.subplots_adjust(left=0.05, right=0.89, top=0.93, bottom=0.08, wspace=0.1, hspace=0.05)
plt.savefig("Compound-Flooding.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [0]:

from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase

fig, ax = plt.subplots(2, 2, figsize=(9, 7.2))

# Fixed color scale range
vmin = 0
vmax = 12
norm = Normalize(vmin=vmin, vmax=vmax)
#cmap = cm.get_cmap('plasma')

blues = plt.cm.Blues

cmap= LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

cd10 = combined_depth_10.where(combined_depth_10 > 0)
cd50 = combined_depth_50.where(combined_depth_50 > 0)
cd100 = combined_depth_100.where(combined_depth_100 > 0)
cd500 = combined_depth_500.where(combined_depth_500 > 0)

# Plot all compound flood depths
cd10.plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd50.plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd100.plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
cd500.plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# Basemaps and styling
for a in ax.flat:
    ctx.add_basemap(a, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)
    a.set_aspect('equal')
    a.set_xlabel('')
    a.set_ylabel('')
    a.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False, right=False, left=False, labelleft=False)

# Titles
titles = ['10-year', '50-year', '100-year', '500-year']
for i, title in enumerate(titles):
    row, col = divmod(i, 2)
    ax[row, col].set_title(title, fontsize=11)

# Shared vertical colorbar
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.7])
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Flood Depths (ft)", fontsize=11)
cb.ax.tick_params(labelsize=11)
cb.outline.set_visible(False)

# Modify the top label of the colorbar
cb.ax.set_yticklabels([f"{vmin}","2", "4", "6", "8", "10", f"$\geq${vmax}"])

# Title
#fig.suptitle('Compound Flood Depths (ft)', fontsize=11, y=0.965)

# Layout
plt.subplots_adjust(left=0.05, right=0.89, top=0.93, bottom=0.08, wspace=0.1, hspace=0.05)
plt.savefig("Compound-Flooding.pdf", dpi=300, bbox_inches="tight")
plt.show()

# Figure 9

In [0]:


fig, ax = plt.subplots(2, 2, figsize=(9, 7.2))

# Colorbar range - converted to meters
vmin, vmax = 0, 4 * 0.3048  # Convert 4 feet to meters (~1.22 m)
norm = Normalize(vmin=vmin, vmax=vmax)

blues = plt.cm.Blues
cmap = LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

# Convert data to meters and plot panels without colorbars
(nt_contribution_10 * 0.3048).plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(nt_contribution_50 * 0.3048).plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(nt_contribution_100 * 0.3048).plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
(nt_contribution_500 * 0.3048).plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)

# Add basemap + cleanup
for a in ax.flat:
    ctx.add_basemap(a, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)
    a.set_aspect('equal')
    a.set_xlabel('')
    a.set_ylabel('')
    a.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False, right=False, left=False, labelleft=False)

# Overlay zone outlines as boundaries for visibility
g10.to_crs(26915).plot(ax=ax[0,0], color='tab:red', linewidth=.45, zorder=10)
g50.to_crs(26915).plot(ax=ax[0,1], color='tab:red', linewidth=.45, zorder=10)
g100.to_crs(26915).plot(ax=ax[1,0], color='tab:red', linewidth=.45, zorder=10)
g500.to_crs(26915).plot(ax=ax[1,1], color='tab:red', linewidth=.45, zorder=10)

# Titles
titles = ['10-year', '50-year', '100-year', '500-year']
for i, title in enumerate(titles):
    row, col = divmod(i, 2)
    ax[row, col].set_title(title, fontsize=11)

# overlay the Amite transect with 0/20/40/60/80 km marks slightly right of center on EACH panel
for a in ax.flat:
    overlay_transect(a, fixed_line, km_ticks=(0,20,40,60,80,100,115),
                     line_lw=.5, dot_size=15, label_fontsize=7,
                     ensure_north_start=False, label_position='right')

# Shared vertical colorbar with explicit tick positions
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.7])
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Contribution to WSE, m", fontsize=11)

# Set explicit tick positions in meters (0, 1, 2, 3, 4 feet converted)
tick_positions = [0, 0.3048, 0.6096, 0.9144, vmax]
cb.set_ticks(tick_positions)
cb.ax.set_yticklabels([
    f"{0:.1f}",
    f"{0.3:.1f}",
    f"{0.6:.1f}",
    f"{0.9:.1f}",
    f"$\\geq${vmax:.1f}"
])
cb.ax.tick_params(labelsize=11)
cb.outline.set_visible(False)

# Final layout adjustments
plt.subplots_adjust(left=0.05, right=0.89, top=0.93, bottom=0.08, wspace=0.1, hspace=0.05)
plt.savefig("Non-tropical-Contributation.pdf", dpi=300, bbox_inches="tight")
plt.show()

In [0]:
import matplotlib.pyplot as plt
import numpy as np
import contextily as ctx
from matplotlib.colors import Normalize
from matplotlib import cm
from matplotlib.patches import Polygon
from matplotlib.colorbar import ColorbarBase

fig, ax = plt.subplots(2, 2, figsize=(9, 7.2))

# Colorbar range
vmin, vmax = 0, 4
norm = Normalize(vmin=vmin, vmax=vmax)
#cmap = cm.get_cmap('plasma')

blues = plt.cm.Blues

cmap= LinearSegmentedColormap.from_list(
    'Blues_dark', blues(np.linspace(0.3, 1.0, 256))
)

# Plot panels without colorbars
nt_contribution_10.plot(ax=ax[0,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
nt_contribution_50.plot(ax=ax[0,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
nt_contribution_100.plot(ax=ax[1,0], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)
nt_contribution_500.plot(ax=ax[1,1], cmap=cmap, norm=norm, add_colorbar=False, rasterized=True)


# Add basemap + cleanup
for a in ax.flat:
    ctx.add_basemap(a, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)
    a.set_aspect('equal')
    a.set_xlabel('')
    a.set_ylabel('')
    a.tick_params(axis='both', which='both', bottom=False, top=False,
                  labelbottom=False, right=False, left=False, labelleft=False)

# Overlay zone outlines as boundaries for visibility
g10.to_crs(26915).plot(ax=ax[0,0], color='tab:red', linewidth=.45, zorder=10)
g50.to_crs(26915).plot(ax=ax[0,1], color='tab:red', linewidth=.45, zorder=10)
g100.to_crs(26915).plot(ax=ax[1,0], color='tab:red', linewidth=.45, zorder=10)
g500.to_crs(26915).plot(ax=ax[1,1], color='tab:red', linewidth=.45, zorder=10)

# Titles
titles = ['10-year', '50-year', '100-year', '500-year']
for i, title in enumerate(titles):
    row, col = divmod(i, 2)
    ax[row, col].set_title(title, fontsize=11)

# overlay the Amite transect with 0/20/40/60/80 km marks slightly right of center on EACH panel
# (set ensure_north_start=True if you want "0" forced to the top/north)
for a in ax.flat:
    overlay_transect(a, fixed_line, km_ticks=(0,20,40,60,80,100,115),
                     line_lw=.5, dot_size=15, label_fontsize=7,
                     ensure_north_start=False, label_position='right')

# Shared vertical colorbar with triangle tips
cbar_ax = fig.add_axes([0.915, 0.16, 0.02, 0.7])
cb = ColorbarBase(cbar_ax, cmap=cmap, norm=norm, orientation='vertical')
cb.set_label("Contribution to WSE (ft)", fontsize=11)
ticks = [vmin, 1, 2, 3, vmax]
cb.set_ticks(ticks)
cb.ax.set_yticklabels([f"{vmin}","1","2","3",f"$\\geq${vmax}"])
cb.ax.tick_params(labelsize=11)
cb.outline.set_visible(False)


# Main title
#fig.suptitle("Contribution of NonTC to Combined flood depths (ft)", fontsize=11, y=0.965)

# Final layout adjustments
plt.subplots_adjust(left=0.05, right=0.89, top=0.93, bottom=0.08, wspace=0.1, hspace=0.05)
plt.savefig("Non-tropical-Contributation.pdf", dpi=300, bbox_inches="tight")
plt.show()

## Examine Transect of NT contribution

In [0]:
from scipy.ndimage import gaussian_filter

# Apply Gaussian smoothing to the values
smoothed_values_10 = gaussian_filter(values_10, sigma=1)

# Use smoothed_values_10 for plotting

In [0]:
from scipy.ndimage import gaussian_filter
#I couldn't figure out how to fix order issues with the amite line in arcpro, so we do it here
fixed_line = ensure_connected_linestring(amite_line.geometry[0])
fixed_line_gdf = gpd.GeoDataFrame({'geometry': [fixed_line]}, crs = 'EPSG:26915')

distances, values_10 = extract_raster_along_line(nt_contribution_10, fixed_line_gdf)
smoothed_values_10 = gaussian_filter(values_10, sigma=5)
distances, values_50 = extract_raster_along_line(nt_contribution_50, fixed_line_gdf)
smoothed_values_50 = gaussian_filter(values_50, sigma=5)
distances, values_100 = extract_raster_along_line(nt_contribution_100, fixed_line_gdf)
smoothed_values_100 = gaussian_filter(values_100, sigma=5)
distances, values_500 = extract_raster_along_line(nt_contribution_500, fixed_line_gdf)
smoothed_values_500 = gaussian_filter(values_500, sigma=5)

distances, compound_10_values = extract_raster_along_line(compound_10, fixed_line_gdf)
distances, compound_50_values = extract_raster_along_line(compound_50, fixed_line_gdf)
distances, compound_100_values = extract_raster_along_line(compound_100, fixed_line_gdf)
distances, compound_500_values = extract_raster_along_line(compound_500, fixed_line_gdf)

distances, pluvial_surge_riverine_10_values = extract_raster_along_line(pluvial_surge_riverine_10, fixed_line_gdf)
distances, pluvial_surge_riverine_50_values = extract_raster_along_line(pluvial_surge_riverine_50, fixed_line_gdf)
distances, pluvial_surge_riverine_100_values = extract_raster_along_line(pluvial_surge_riverine_100, fixed_line_gdf)
distances, pluvial_surge_riverine_500_values = extract_raster_along_line(pluvial_surge_riverine_500, fixed_line_gdf)

fig = plt.figure(figsize=(7.5, 12))  # academic width ~7.5 inches

gs = fig.add_gridspec(5, 2)
ax1 = fig.add_subplot(gs[1, 0])
ax2 = fig.add_subplot(gs[2, 0])
ax3 = fig.add_subplot(gs[3, 0])
ax4 = fig.add_subplot(gs[4, 0])
ax5 = fig.add_subplot(gs[1, 1])
ax6 = fig.add_subplot(gs[2, 1])
ax7 = fig.add_subplot(gs[3, 1])
ax8 = fig.add_subplot(gs[4, 1])
ax9 = fig.add_subplot(gs[0, :])  # overview map

# Plot lines - Left column: solid (Compound), dashed (Max of 3)
ax1.plot(distances/1000, compound_10_values, linestyle='-', color='black')
ax1.plot(distances/1000, pluvial_surge_riverine_10_values, linestyle='--', color='black')
ax2.plot(distances/1000, compound_50_values, linestyle='-', color='black')
ax2.plot(distances/1000, pluvial_surge_riverine_50_values, linestyle='--', color='black')
ax3.plot(distances/1000, compound_100_values, linestyle='-', color='black')
ax3.plot(distances/1000, pluvial_surge_riverine_100_values, linestyle='--', color='black')
ax4.plot(distances/1000, compound_500_values, linestyle='-', color='black')
ax4.plot(distances/1000, pluvial_surge_riverine_500_values, linestyle='--', color='black')

# Right column: Difference plots (black)
ax5.plot(distances/1000, smoothed_values_10, color='black')
ax6.plot(distances/1000, smoothed_values_50, color='black')
ax7.plot(distances/1000, smoothed_values_100, color='black')
ax8.plot(distances/1000, smoothed_values_500, color='black')

# Set y-axis limits for right column
for ax in [ax5, ax6, ax7, ax8]:
    ax.set_ylim(0, 4)

# Set x-axis ticks for right column to every 10 km
for ax in [ax5, ax6, ax7, ax8]:
    ax.set_xticks(np.arange(0, distances.max()/1000 + 10, 10))

# Labels
for ax in [ax1, ax2, ax3, ax4]:
    ax.set_ylabel('WSE (ft)', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)

for ax in [ax5, ax6, ax7, ax8]:
    ax.set_ylabel('WSE Difference (ft)', fontsize=11)
    ax.set_xlabel('Distance (km)', fontsize=11)
    ax.tick_params(labelsize=11)

# Titles
ax1.set_title('10-year', fontsize=11)
ax2.set_title('50-year', fontsize=11)
ax3.set_title('100-year', fontsize=11)
ax4.set_title('500-year', fontsize=11)
ax5.set_title('10-year', fontsize=11)
ax6.set_title('50-year', fontsize=11)
ax7.set_title('100-year', fontsize=11)
ax8.set_title('500-year', fontsize=11)

#fig.suptitle('Difference between compound flood depths and max of 3 individual drivers along Amite transect', fontsize=11)

# Top map
ax9.tick_params(axis='both', which='both', bottom=False, top=False,
                labelbottom=False, right=False, left=False, labelleft=False)
fixed_line_gdf.plot(ax=ax9)
domain_gdf.exterior.plot(ax=ax9)
ctx.add_basemap(ax9, crs="EPSG:26915", source=ctx.providers.CartoDB.Positron)

# Custom legend for left column
from matplotlib.lines import Line2D

fig.subplots_adjust(top=0.96, hspace=0.6, wspace=0.3)

# Custom line styles
legend_elements_left = [
    Line2D([0], [0], linestyle='-', color='black', label='Compounded Flood Elevation'),
    Line2D([0], [0], linestyle='--', color='black', label='Max of Pluvial, Fluvial, and Coastal Drivers')
]

from matplotlib.patches import Patch  # add this import at the top if not already present

legend_elements_right = [
    Patch(facecolor='none', edgecolor='none', label='Flood Depth Difference of Compound Flood\nand Max of Pluvial, Fluvial, and Coastal Drivers')
]

#Axes for placing column legends
ax_legend_left = fig.add_axes([0.07, 0.79, 0.42, 0.05])   # [left, bottom, width, height]
ax_legend_right = fig.add_axes([0.52, 0.79, 0.42, 0.05])

# Turn off everything for dummy axes
for ax in [ax_legend_left, ax_legend_right]:
    ax.axis('off')

# Left column legend
ax_legend_left.legend(
    handles=legend_elements_left,
    loc='center',
    fontsize=10,
    frameon=False
)

# Right column legend
ax_legend_right.legend(
    handles=legend_elements_right,
    loc='center',
    fontsize=10,
    frameon=False
)